<a href="https://colab.research.google.com/github/kylianmbappe9/Xanadu-CodeBook/blob/main/Heat_Model_In_Conduction_Convection_in_Multiple_Conductances_using_Machine_Learning_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers



In [8]:
def physics_model_3d(T, dx, dy, dz, dt, alpha=0.01):
    d2T_dx2 = (np.roll(T, 1, axis=2) - 2 * T + np.roll(T, -1, axis=2)) / (dx**2)
    d2T_dy2 = (np.roll(T, 1, axis=1) - 2 * T + np.roll(T, -1, axis=1)) / (dy**2)
    d2T_dz2 = (np.roll(T, 1, axis=0) - 2 * T + np.roll(T, -1, axis=0)) / (dz**2)
    T_new = T + alpha * dt * (d2T_dx2 + d2T_dy2 + d2T_dz2)
    return T_new


In [9]:
def build_generator(input_shape):
    model = tf.keras.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(np.prod(input_shape), activation='linear'),
        layers.Reshape(input_shape)
    ])
    return model

In [10]:
def train_model(epochs, initial_temperature, dx, dy, dz, dt, lambda_physics=1.0):
    grid_size = initial_temperature.shape
    physics_T = initial_temperature.copy()
    generator = build_generator(grid_size)
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            predicted_T = generator(np.expand_dims(physics_T, axis=0), training=True)[0]

            # Data loss (MSE between predicted and actual temperature)
            #MSE is mean squared error AND WE JUSt want to
            #differentiate between the mean squared error of ML and PIML and ensure its smaller
            actual_T = physics_model_3d(physics_T, dx, dy, dz, dt)
            data_loss = tf.reduce_mean(tf.square(predicted_T - actual_T))

            # Physics loss (checking if predicted_T satisfies the 3D heat equation)
            d2T_dx2 = (tf.roll(predicted_T, 1, axis=2) - 2 * predicted_T + tf.roll(predicted_T, -1, axis=2)) / (dx**2)
            d2T_dy2 = (tf.roll(predicted_T, 1, axis=1) - 2 * predicted_T + tf.roll(predicted_T, -1, axis=1)) / (dy**2)
            d2T_dz2 = (tf.roll(predicted_T, 1, axis=0) - 2 * predicted_T + tf.roll(predicted_T, -1, axis=0)) / (dz**2)
            physics_residual = tf.abs((predicted_T - actual_T) / dt - (d2T_dx2 + d2T_dy2 + d2T_dz2))
            physics_loss = tf.reduce_mean(physics_residual)

            # Total loss (ML loss + weighted physics loss)
            total_loss = data_loss + lambda_physics * physics_loss

        # Compute gradients and apply updates
        grads = tape.gradient(total_loss, generator.trainable_variables)
        optimizer.apply_gradients(zip(grads, generator.trainable_variables))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Data Loss: {data_loss.numpy()}, Physics Loss: {physics_loss.numpy()}, Total Loss: {total_loss.numpy()}")

        # Update the physics model temperature (real behavior)
        physics_T = actual_T


In [11]:
grid_size = (50, 50, 50)  # 3D grid size
dx, dy, dz = 1.0, 1.0, 1.0  # Spatial steps
dt = 0.01  # Time step
initial_temperature = np.zeros(grid_size)
initial_temperature[25, 25, 25] = 100  # Hot spot in the center of the 3D grid

In [12]:
train_model(epochs=1000, initial_temperature=initial_temperature, dx=dx, dy=dy, dz=dz, dt=dt)

Epoch 0, Data Loss: 0.07997702807188034, Physics Loss: 0.8112882375717163, Total Loss: 0.8912652730941772
Epoch 100, Data Loss: 0.0707937553524971, Physics Loss: 0.095188669860363, Total Loss: 0.1659824252128601
Epoch 200, Data Loss: 0.06284353882074356, Physics Loss: 0.09494806826114655, Total Loss: 0.1577916145324707
Epoch 300, Data Loss: 0.05588946491479874, Physics Loss: 0.09057974815368652, Total Loss: 0.14646920561790466
Epoch 400, Data Loss: 0.049794040620326996, Physics Loss: 0.09212061017751694, Total Loss: 0.14191465079784393
Epoch 500, Data Loss: 0.04443962872028351, Physics Loss: 0.08815043419599533, Total Loss: 0.13259005546569824
Epoch 600, Data Loss: 0.03972594812512398, Physics Loss: 0.08646100759506226, Total Loss: 0.12618695199489594
Epoch 700, Data Loss: 0.03556735813617706, Physics Loss: 0.08487284183502197, Total Loss: 0.12044019997119904
Epoch 800, Data Loss: 0.0318906232714653, Physics Loss: 0.08334868401288986, Total Loss: 0.11523930728435516
Epoch 900, Data Los